# ----------声明常量----------

In [1]:
# Hongjun Wu
# 20180723
# A script that recommends stock based on data and conditions given.

# Import Statement
from selenium import webdriver
from bs4 import BeautifulSoup
from decimal import Decimal
from selenium.common.exceptions import ElementNotVisibleException
import turicreate as tc
import datetime
from zhon.hanzi import punctuation
import os

# 自然语言识别框架
import pyltp
from pyltp import SentenceSplitter
from pyltp import Segmentor
from pyltp import Postagger
from pyltp import NamedEntityRecognizer
from pyltp import Parser
from pyltp import SementicRoleLabeller
LTP_DATA_DIR = '../../../LTP_data_v3.4.0/'  # ltp模型目录的路径
pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')  # 词性标注模型路径，模型名称为`pos.model`
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`
ner_model_path = os.path.join(LTP_DATA_DIR, 'ner.model')  # 命名实体识别模型路径，模型名称为`pos.model`
par_model_path = os.path.join(LTP_DATA_DIR, 'parser.model')  # 依存句法分析模型路径，模型名称为`parser.model`
srl_model_path = os.path.join(LTP_DATA_DIR, 'srl')  # 语义角色标注模型目录路径，模型目录为`srl`。注意该模型路径是一个目录，而不是一个文件。

# 储存记录文件的根目录
fileRoot = './SelectedData'

# 存储四个给定变量的地方
var_list= []

# 代码对应网址的字典
code_dict = {'a': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04471',
             'b': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04931',
             'c': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK05231',
             'd': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK06991',
             'e': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK07271',
             'f': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04741',
             'g': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK05381',
             'h': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK07311',
             'i': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04781',
             'j': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04791',
             'k': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04561',
             'l': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK07331',
             'm': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04241',
             'n': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK07321',
             'o': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK07361',
             'p': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04801',
             'q': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04761',
             'r': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK07381',
             's': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04381',
             't': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK05381',
             'u': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04241',
             'v': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK07361',
             'w': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04561',
             'x': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK09101',
             'y': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK07401',
             'z': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04221',
             'aa': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04541',
             'bb': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK07391',
             'cc': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04571',
             'dd': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04641',
             'ee': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK05451',
             'ff': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK07281',
             'gg': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04851',
             'hh': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK07291',
             'ii': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK07351',
             'jj': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04511',
             'kk': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04751',
             'll': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04811',
             'mm': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK07251',
             'nn': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK07391',
             'oo': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK07261',
             'pp': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04731',
             'qq': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04501',
             'rr': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04281',
             'ss': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04701',
             'tt': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04571',
             'uu': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK07311',
             'vv': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04291',
             'ww': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK07301',
             'xx': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK05391',
             'yy': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK05371',
             'zz': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04861',
             'aaa': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04841',
             'bbb': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK07371',
             'ccc': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04471',
             'ddd': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04591',
             'eee': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04651',
             'fff': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK07331',
             'ggg': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04331',
             'hhh': 'http://quote.eastmoney.com/center/boardlist.html#boards-BK04771'}

name_dict = {'电子信息': 'a',
             '新能源':'b',
             '新材料':'c',
             '全息技术':'d',
             '医疗行业':'e',
             '保险':'f',
             '化工行业':'g',
             '化肥行业':'h',
             '有色金属':'i',
             '钢铁行业':'j',
             '家电行业':'k',
             '包装材料':'l',
             '水泥建材':'m',
             '贵金属':'n',
             '电信运营':'o',
             '航天航空':'p',
             '木业家具':'q',
             '多元金融':'r',
             '食品饮料':'s',
             '化工行业':'t',
             '水泥建材':'u',
             '电信运营':'v',
             '家电行业':'w',
             '专用设备':'x',
             '文教休闲':'y',
             '交运物流':'z',
             '塑胶制品':'aa',
             '金属制品':'bb',
             '输配电气':'cc',
             '石油行业':'dd',
             '机械行业':'ee',
             '环保工程':'ff',
             '旅游酒店':'gg',
             '船舶制造':'hh',
             '安防设备':'ii',
             '房地产':'jj',
             '银行':'kk',
             '汽车行业':'ll',
             '装修装饰':'mm',
             '金属制品':'nn',
             '园林工程':'oo',
             '券商信托':'pp',
             '港口水运':'qq',
             '电力行业':'rr',
             '造纸印刷':'ss',
             '输配电气':'tt',
             '化肥行业':'uu',
             '交运设备':'vv',
             '农药兽药':'ww',
             '综合行业':'xx',
             '材料行业':'yy',
             '文化传媒':'zz',
             '国际贸易':'aaa',
             '软件服务':'bbb',
             '电子信息':'ccc',
             '电子元件':'ddd',
             '医药制造':'eee',
             '包装材料':'fff',
             '农牧饲渔':'ggg',
             '酿酒行业':'hhh'}

codename_dict = {'a': '电子信息',
             'b': '新能源',
             'c': '新材料',
             'd': '全息技术',
             'e': '医疗行业',
             'f': '保险',
             'g': '化工行业',
             'h': '化肥行业',
             'i': '有色金属',
             'j': '钢铁行业',
             'k': '家电行业',
             'l': '包装材料',
             'm': '水泥建材',
             'n': '贵金属',
             'o': '电信运营',
             'p': '航天航空',
             'q': '木业家具',
             'r': '多元金融',
             's': '食品饮料',
             't':'化工行业',
             'u':'水泥建材',
             'v':'电信运营',
             'w':'家电行业',
             'x':'专用设备',
             'y':'文教休闲',
             'z':'交运物流',
             'aa':'塑胶制品',
             'bb':'金属制品',
             'cc':'输配电气',
             'dd':'石油行业',
             'ee':'机械行业',
             'ff':'环保工程',
             'gg':'旅游酒店',
             'hh':'船舶制造',
             'ii':'安防设备',
             'jj':'房地产',
             'kk':'银行',
             'll':'汽车行业',
             'mm':'装修装饰',
             'nn':'金属制品',
             'oo':'园林工程',
             'pp':'券商信托',
             'qq':'港口水运',
             'rr':'电力行业',
             'ss':'造纸印刷',
             'tt':'输配电气',
             'uu':'化肥行业',
             'vv':'交运设备',
             'ww':'农药兽药',
             'xx':'综合行业',
             'yy':'材料行业',
             'zz':'文化传媒',
             'aaa':'国际贸易',
             'bbb':'软件服务',
             'ccc':'电子信息',
             'ddd':'电子元件',
             'eee':'医药制造',
             'fff':'包装材料',
             'ggg':'农牧饲渔',
             'hhh':'酿酒行业'}

bankuai_weight_dict = {'软件服务': 10,
                       '航天航空': 10,  # 再议
                       '新能源': 9,  # 再议
                       '新材料': 9,
                       '材料行业': 9,  # 再议
                       '电子信息': 8,
                       '有色金属': 4,
                       '?': 3,  # 这啥
                       '钢铁行业': 2}

# ----------功能性函数----------

In [2]:
# 功能性函数

# 欢迎界面
def greetings():
    print('+-------------------欢迎界面-------------------+')
    print('| 请选择您要执行的操作:                          |')
    print('| 1. 输入a来分析指定板块                         |')
    print('| 2. 输入s来搜寻板块代码                         |')
    print('| 3. 输入l来显示所有代码                         |')
    print('| 4. 输入x来显示所有选股                         |')
    print('| 5. 输入j来加载以前分析完成股票数据               |')
    print('| 6. 输入r来加载分析后股票按分数排序               |')
    print('| 7. 输入n来加载以前分析完成新闻数据               |')
    print('|                 （输入q退出）                 |')
    print('+---------------------------------------------+')
    choice = str(input('命令:'))
    return choice


# 汉字转数字
def smartMultiply(string):
    if string[len(string) - 1:len(string)] == '万':
        string = Decimal(string[0:len(string) - 1])
        string = float(string) * 10000
    elif string[len(string) - 1:len(string)] == '亿':
        string = Decimal(string[0:len(string) - 1])
        string = float(string) * 100000000
    elif string[len(string) - 1:len(string)] == '%':
        string = Decimal(string[0:len(string) - 1])
        string = float(string) * 0.01
    else:
        string = float(string)
    return string


# 数字转汉字
def smartDivide(number, isPercent):
    if isPercent:
        return str(1.0 * number / 0.01) + '%'
    elif number > 100000000:
        return str(1.0 * number / 100000000) + '亿'
    elif number > 10000:
        return str(1.0 * number / 10000) + '万'
    else:
        return str(number)


# searchCode() 用于搜寻股票代码
def searchCode():
    isDone = False
    print('+--------------输入想要查找的板块名--------------+')
    while isDone == False:
        user_input = str(input('搜索板块名:'))
        if user_input == 'quit':
            print('用户已取消操作！')
            break
        try:
            print(user_input + '代码是：' + name_dict[user_input])
            isDone = True
        except KeyError:
            print('板块不存在!')
    # 重启UI (待议)
    user_interface(SELECTED_DATA)
    
    
# printAllBankuai()
def printAllBankuai():
    print('----------------以下是所有板块代码----------------')
    element_list = [(k, codename_dict[k]) for k in sorted(codename_dict.keys())]
    counter = 0
    while counter < len(element_list):
        print(element_list[counter][0] + ' - ' + element_list[counter][1])
        counter += 1
    print('-----------------------------------------------')
    # 重启UI
    user_interface(SELECTED_DATA)
    
    
# 显示所有的选股
def showAllSelected(xuangu):
    # print(SELECTED_DATA)
    if xuangu[0]['name'] != '数据不存在':
        # xuangu.show()
        print(xuangu)
    else:
        print('没有分析完成的数据！')
    user_interface(xuangu)
    return xuangu  # 这个非常不确定
    
    
# 用于加载SFrame (也许需要写一个return 待议)
def parseSFrame():
    fileName = input('输入文件名：')
    filePath = './SelectedData/' + fileName + '/'
    loaded_data = tc.SFrame(data=filePath)
    answer = input('要显示载入的SFrame吗？(y/n)')
    if answer == 'y':
        # loaded_data.materialize()
        print(loaded_data)
        # loaded_data.show()
    return loaded_data
    
    
# Literally show SFrame（大概没啥用）
def showSFrame(SFrame):
    SFrame.show()
    
# 初步筛选分析程序
def analyze_stock(SFrame):
    SFrame = analysis_turnover_rate(SFrame, var_list[0])
    SFrame = analysis_volume_rate(SFrame, var_list[1])
    return SFrame


# 返回所有换手率大于5%的行
def analysis_turnover_rate(SFrame, turnover_rate):
    return SFrame[SFrame['turnover_rate'] > turnover_rate]


# 返回所有量比大于30%的行
def analysis_volume_rate(SFrame, volume_rate):
    return SFrame[ SFrame['volume_rate'] > volume_rate]


# 一个从页面获取页数的函数
def getPageNumber(bs):
    all_buttons = bs.findAll(class_="paginate_button")
    if len(all_buttons) == 2:
        return 1  # 处理只有一页的情况
    else:
        return len(all_buttons) - 2  # 下一页和Go按钮

    
# 把数据中的 - 改为 0
def noSlash(str):
    if str == '-':
        return '0'
    else:
        return str
    
    
# 用于保存SFrame
def saveSFrame(SFrame, fileRoot):
    # 保存数据
    date = '20' + str(datetime.datetime.now().strftime("%y%m%d%p"))
    filepath = fileRoot + '/' + str(date) + '/'
    SFrame.save(filepath)
    print('成功保存数据文件！数据路径：' + filepath)

    # 打印时间戳
    print('程序运行时间戳：20'
          + str(datetime.datetime.now().strftime("%y")) + '年'
          + str(datetime.datetime.now().strftime("%m")) + '月'
          + str(datetime.datetime.now().strftime("%d")) + '日'
          + str(datetime.datetime.now().strftime("%H")) + '时'
          + str(datetime.datetime.now().strftime("%M")) + '分'
          + str(datetime.datetime.now().strftime("%S")) + '秒')
    
# 用于保留x位小数
def cutDecimal(x, string):
    return string[0:x+2]

def balanceSArray(number, SArray_sum):
    return number / Sarray_SUM

# 用于执行normalize
def normalizeColumns(SFrame):
    waitingList = ['PERation', 'amplitude', 'change', 'close',
                   'income_increase', 'profit_increase', 'percent_chg', 
                   'turn_volume', 'turnover_rate', 'volume', 'volume_rate']
    for element in waitingList:
        sumSArray = sum(SFrame[element])
        # print('sumSArray', sumSArray)
        newSArrayName = 'balanced_' + element
        if sumSArray != 0:
            SFrame[newSArrayName] = SFrame[element].apply(lambda x: x / sumSArray)
        else:
            SFrame[newSArrayName] = SFrame[element].apply(lambda x: x * 0)
        
    return SFrame


# 用于返回指定板块的加分项目
def bankuaiScore(bankuai):
    if bankuai in bankuai_weight_dict.keys():
        return bankuai_weight_dict[bankuai]
    else:
        return 1
    
    
# 用于计算每只股票的score
def makeSCore(SFrameRow):
    score = 0
    score += SFrameRow['balanced_profit_increase'] * 0.2  # 净利润
    score += SFrameRow['balanced_income_increase'] * 0.1  # 收入
    score += SFrameRow['balanced_turnover_rate'] * 0.1  # 换手率
    score += SFrameRow['balanced_amplitude'] * 0.1  # 量比
    score += SFrameRow['balanced_PERation'] * 0.1  # 市盈率
    score += bankuaiScore(SFrameRow['bankuai']) * 0.1  # 行业
    
    return score * 100


# 用于显示排名和分数
def sortSelectStocks(SFrame, columnString):
    if columnString == '':
        columnString = 'name,code,score'
    
    columnList = []
    index = 0
    firstIndex = 0
    while index < len(columnString):
        if columnString[index] == ',':
            columnList.append(columnString[firstIndex: index])
            firstIndex = index + 1
        elif index == len(columnString) - 1:
            columnList.append(columnString[firstIndex: index + 1])
        index += 1
        
    # print(columnList)
    visual_SFrame = SFrame[columnList]
    return visual_SFrame
    

# ----------执行型函数----------

In [3]:
def user_interface(xuangu):
    # print('我正在进入user_interface', len(xuangu), xuangu['name'])
    choice = greetings()
    if choice == 'a':
        done_xuangu = inputCode(xuangu)
        done_xuangu = done_xuangu[1:len(done_xuangu)]
        saveSFrame(done_xuangu, fileRoot)
        # print('我在user_interface', done_xuangu)
        return done_xuangu
    elif choice == 's':
        searchCode()
    elif choice == 'l':
        printAllBankuai()
    elif choice == 'x':
        xuangu = showAllSelected(xuangu)
    elif choice == 'j':
        xuangu = parseSFrame()
        user_interface(xuangu)
    elif choice == 'r':
        selected_stocks = sortSelectStocks(SELECTED_DATA, input('输入需要显示的列名用,隔开不要空格'))
        print(selected_stocks)
    elif choice == 'q':
        exit
    else:
        user_interface(xuangu)  # 这边这个SELECTED_DATA是真的SELECTED_DATA吗 我认为不是
    return xuangu


# 分析板块主程序
def inputCode(xuangu):
    # print('我正在进入inputCode', len(xuangu), xuangu['name'])
    print('+-------------------分析板块--------------------+')
    print('在下面输入控制变量：                               ')
    turnover_rate = input('换手率大于（默认0.05对应5%）：')
    if turnover_rate == '':
        print('换手率设为默认0.05！')
        turnover_rate = 0.05
    var_list.append(float(turnover_rate))

    volume_rate= input('量比大于（默认0.3对应30%）：')
    if volume_rate == '':
        print('量比设为默认0.3！')
        volume_rate = 0.3
    var_list.append(float(volume_rate))

    income_rate = input('营业收入大于（默认0.3对应30%）：')
    if income_rate == '':
        print('营业收入设为默认0.3！')
        income_rate = 0.3
    var_list.append(float(income_rate))

    benefit_rate = input('净利润大于（默认0.3对应30%）:')
    if benefit_rate == '':
        print('净利润设为默认0.3！')
        benefit_rate = 0.3
    var_list.append(float(benefit_rate))

    print('+----------------------------------------------+')
    print('|输入板块代号（输入quit退出至主页面, all分析所有板块）|')
    print('-----------------------------------------------')
    code_name = str(input('代号:'))
    if code_name == 'all':
        for each_bk in code_dict:
            bk_name = codename_dict[each_bk]
            # xuangu = xuangu.append(makeRecommend(xuangu, code_dict[each_bk], bk_name))
            xuangu = makeRecommend(xuangu, code_dict[each_bk], bk_name)
    elif code_name == 'quit':
        user_interface(xuangu)
    else:
        xuangu = makeRecommend(xuangu, code_dict[code_name], codename_dict[code_name])

    # print(xuangu)
    # showSFrame(xuangu)
    # print('我正在离开inputCode', len(xuangu), xuangu['name'])
    return xuangu


# 用于完成筛选代码的全过程
def makeRecommend(xuangu, url, bk_name):
    # print('我正在进入makeRecommend', len(xuangu), xuangu['name'])
    # 创建四个空SFrame，以占位行开头
    all_data = tc.SFrame({'code': ['000000'], 'name': ['哔哩哔哩'],
                          'close': [0.0], 'percent_chg': [0.0],
                          'change': [0.0], 'volume': [0.0], 'turn_volume': [0.0], 'amplitude': [0.0],
                          'high': [0.0], 'low': [0.0],
                          'now_open': [0.0], 'previous_close': [0.0], 'volume_rate': [0.0],
                          'turnover_rate': [0.0], 'report_url': ['http://www.bilibili.com'], 'PERation': [0.0]})

    # 获取信息
    all_data = makeData(url, all_data)

    # 初步筛选
    analyze_data = analyze_stock(all_data)

    # 最终推荐
    print('---------------------' + bk_name + '---------------------')
    xuangu = recommendStock(xuangu, analyze_data, bk_name)
    # print('我正在离开makeRecommend', len(xuangu), xuangu['name'])
    return xuangu

# 自动处理粗数据的主程序
def makeData(url, SFrame):
    browser = webdriver.Chrome()  # Get local session of chrome
    # url = search_area[topic]  # Example: '电子信息'
    browser.get(url)  # Load page
    browser.implicitly_wait(20)  # 智能等待20秒

    # 第一次访问时判定菜单数量来决定浏览多少次表格
    bs = BeautifulSoup(browser.page_source, "lxml")
    page_number = getPageNumber(bs)

    # 循环浏览页面直到搜集完毕所有table
    counter = 0
    while counter < page_number:
        SFrame = grabData(bs, SFrame)
        try:
            browser.find_element_by_id('main-table_next').click()
        except ElementNotVisibleException:
            print('Warning: 无法获得某些破损的数据.')
        bs = BeautifulSoup(browser.page_source, "lxml")
        counter += 1

    SFrame = SFrame[1:len(SFrame)]  # 删掉占位符
    SFrame = SFrame.unique()
    return SFrame

# 从一个静态BeautifulSoup页面解析表格并存储进SFrame
def grabData(bs, SFrame):
    # 解出表格
    table = bs.findAll(role='row')
    table = table[7: len(table) - 1]
    # 分析每个表格
    counter = 0

    while counter < len(table):
        row_sframe = tc.SFrame({'code': [str(table[counter].find(class_=' listview-col-Code').string)],
                                'name': [str(table[counter].find(class_=' listview-col-Name').string)],
                                'close': [smartMultiply(noSlash(table[counter].find(class_=' listview-col-Close').string))],
                                'percent_chg': [smartMultiply(noSlash(table[counter].find(class_='listview-col-ChangePercent sorting_1').string))],
                                'change': [smartMultiply(noSlash(table[counter].find(class_=' listview-col-Change').string))],
                                'volume': [smartMultiply(noSlash(table[counter].find(class_=' listview-col-Volume').string))],
                                'turn_volume': [
                                    smartMultiply(noSlash(table[counter].find(class_=' listview-col-Amount').string))],
                                'amplitude': [
                                    smartMultiply(noSlash(table[counter].find(class_=' listview-col-Amplitude').string))],
                                'high': [smartMultiply(noSlash(table[counter].find(class_=' listview-col-High').string))],
                                'low': [smartMultiply(noSlash(table[counter].find(class_=' listview-col-Low').string))],
                                'now_open': [smartMultiply(noSlash(table[counter].find(class_=' listview-col-Open').string))],
                                'previous_close': [
                                    smartMultiply(noSlash(table[counter].find(class_=' listview-col-PreviousClose').string))],
                                'volume_rate': [
                                    smartMultiply(noSlash(table[counter].find(class_=' listview-col-VolumeRate').string))],
                                'turnover_rate': [
                                    smartMultiply(noSlash(table[counter].find(class_=' listview-col-TurnoverRate').string))],
                                'report_url': [
                                    'http://emweb.securities.eastmoney.com/f10_v2/FinanceAnalysis.aspx?type=web&code=sz' +
                                    table[counter].find(class_=' listview-col-Code').string + '#lrb-0'],
                                'PERation':[
                                    smartMultiply(noSlash(table[counter].find(class_=' listview-col-PERation').string))]
                                })
        counter += 1
        SFrame = SFrame.append(row_sframe)

    return SFrame


# 打印选中的股票
def recommendStock(xuangu, SFrame, bankuai):
    # print('我正在进入recommendStock', len(xuangu), xuangu['name'])
    income_limit = var_list[2]
    profit_limit = var_list[3]
    counter = 0
    while counter < len(SFrame):
        result_list = getReport(SFrame[counter], bankuai, SFrame[counter], income_limit, profit_limit)
        if result_list[0]:
            print('股票名称：' + SFrame[counter]['name'])
            print('股票代码：' + SFrame[counter]['code'])
            print('成交量：' + smartDivide(SFrame[counter]['volume'], False))
            print('成交额：' + smartDivide(SFrame[counter]['turn_volume'], False))
            print('成交量比增幅：' + smartDivide(float(cutDecimal(4, str(SFrame[counter]['amplitude']))), True))
            print('换手率：' + smartDivide(float(cutDecimal(4, str(SFrame[counter]['turnover_rate']))), True))
            print('营业总收入增长', smartDivide(float(cutDecimal(4, str(result_list[2]))), True))
            print('净利润增长', smartDivide(float(cutDecimal(4, str(result_list[3]))), True))
            print('-------------------------------------------------')
            # print(result_list[1])
            xuangu = xuangu.append(result_list[1])
        counter += 1
    # print('我正在离开recommendStock', len(xuangu), xuangu['name'])
    return xuangu
        
        
# 查找报表
def getReport(SFrame, bankuai, row, income_limit, profit_limit):
    # 爬取网站源代码
    url = row['report_url']
    browser = webdriver.Chrome()  # Get local session of chrome
    browser.get(url)  # Load page
    soup = BeautifulSoup(browser.page_source, "lxml")
    browser.close()


    # 检查是否报表模板错误
    check_report = soup.findAll(id='stock_full_name123')
    if len(check_report) != 0:
        keyword = check_report[0]['value']
        if keyword == '- - - -':  # 纠正SFrame
            temp_row = row  # 解决无法显示上证股票报表的问题
            temp_row['report_url'] = temp_row['report_url'][0:80] + 'sh' + temp_row['report_url'][82:94]
            return getReport(SFrame, bankuai, temp_row, income_limit, profit_limit)

    # 粗加工数据
    ulist = []
    trs = soup.find_all('tr')
    for tr in trs:
        ui = []
        for td in tr:
            ui.append(td.string)
        ulist.append(ui)

    # 提取营业额和净利润
    income_increase = 0
    profit_increase = 0
    for element in ulist:
        if ('营业总收入' in element):
            income_data_list = element
            now_data = smartMultiply(income_data_list[3])
            past_data = smartMultiply(income_data_list[11])
            income_increase = (now_data - past_data) / past_data
        elif ('净利润' in element):
            profit_data_list = element
            now_data = smartMultiply(profit_data_list[3])
            past_data = smartMultiply(profit_data_list[11])
            profit_increase = (now_data - past_data) / past_data

    # if income_increase > income_limit and profit_increase > profit_limit:
    new_row = tc.SFrame({'code': [row['code']], 'name': [row['name']], 'bankuai': [bankuai],
                               'close': [row['close']], 'percent_chg': [row['percent_chg']], 'change': [row['change']],
                               'volume': [row['volume']], 'turn_volume': [row['turn_volume']],
                               'amplitude': [row['amplitude']], 'volume_rate': [row['volume_rate']], 'turnover_rate': [row['turnover_rate']],
                               'news_url': [''], 'income_increase': [income_increase],
                               'profit_increase': [profit_increase],'PERation': [row['PERation']]})
        
    result = [income_increase > income_limit and profit_increase > profit_limit, new_row, income_increase, profit_increase]
    # print(bankuai,row['name'], result)  # Debug
    return result

# ----------执行命令----------

In [4]:
# 新建精选数据库
SELECTED_DATA = tc.SFrame({'code': ['000000'], 'name': ['数据不存在'],'bankuai': ['二次元'],
                           'close': [0.0], 'percent_chg': [0.0],'change': [0.0],
                           'volume': [0.0], 'turn_volume': [0.0],
                           'amplitude': [0.0],'volume_rate': [0.0],
                           'turnover_rate': [0.0],
                           'news_url': ['http://www.bilibili.com'], 
                           'income_increase': [0.0], 'profit_increase': [0.0], 
                           'PERation': [0.0]})


In [5]:
SELECTED_DATA = user_interface(SELECTED_DATA)
SELECTED_DATA = SELECTED_DATA.unique()  # 查重


SELECTED_DATA = normalizeColumns(SELECTED_DATA)
SELECTED_DATA['score'] = SELECTED_DATA.apply(makeSCore)
SELECTED_DATA = SELECTED_DATA.sort('score', ascending = False)

+-------------------欢迎界面-------------------+
| 请选择您要执行的操作:                          |
| 1. 输入a来分析指定板块                         |
| 2. 输入s来搜寻板块代码                         |
| 3. 输入l来显示所有代码                         |
| 4. 输入x来显示所有选股                         |
| 5. 输入j来加载以前分析完成股票数据               |
| 6. 输入r来加载分析后股票按分数排序               |
| 7. 输入n来加载以前分析完成新闻数据               |
|                 （输入q退出）                 |
+---------------------------------------------+
命令:a
+-------------------分析板块--------------------+
在下面输入控制变量：                               
换手率大于（默认0.05对应5%）：
换手率设为默认0.05！
量比大于（默认0.3对应30%）：
量比设为默认0.3！
营业收入大于（默认0.3对应30%）：
营业收入设为默认0.3！
净利润大于（默认0.3对应30%）:
净利润设为默认0.3！
+----------------------------------------------+
|输入板块代号（输入quit退出至主页面, all分析所有板块）|
-----------------------------------------------
代号:all
---------------------电子信息---------------------
---------------------新能源---------------------
---------------------新材料---------------------
---------------------全息技术--------------------

In [6]:
SELECTED_DATA

Columns:
	PERation	float
	amplitude	float
	bankuai	str
	change	float
	close	float
	code	str
	income_increase	float
	name	str
	news_url	str
	percent_chg	float
	profit_increase	float
	turn_volume	float
	turnover_rate	float
	volume	float
	volume_rate	float
	balanced_PERation	float
	balanced_amplitude	float
	balanced_change	float
	balanced_close	float
	balanced_income_increase	float
	balanced_profit_increase	float
	balanced_percent_chg	float
	balanced_turn_volume	float
	balanced_turnover_rate	float
	balanced_volume	float
	balanced_volume_rate	float
	score	float

Rows: 3

Data:
+----------+----------------------+----------+--------+-------+--------+
| PERation |      amplitude       | bankuai  | change | close |  code  |
+----------+----------------------+----------+--------+-------+--------+
|   0.0    |        0.0707        | 软件服务 |  0.13  |  7.49 | 002072 |
|  56.96   |        0.0633        | 交运物流 |  3.72  | 50.03 | 603713 |
|  23.86   | 0.052300000000000006 | 医药制造 |  0.35  | 12.78 | 600789 |
+----------+----------------------+----------+--------+-------+--------+
+---------------------+----------+----------+-------------+--------------------+
|   income_increase   |   name   | news_url | percent_chg |  profit_increase   |
+---------------------+----------+----------+-------------+--------------------+
|  0.9058558558558558 |  凯瑞德  |          |    0.0177   | 0.6635813456746174 |
|  1.8370259106271125 | 密尔克卫 |          |    0.0803   | 3.6657362177250525 |
| 0.38470031545741323 | 鲁抗医药 |          |    0.0282   | 5.314754098360655  |
+---------------------+----------+----------+-------------+--------------------+
+--------------+---------------+----------+-------------+--------------------+
| turn_volume  | turnover_rate |  volume  | volume_rate | balanced_PERation  |
+--------------+---------------+----------+-------------+--------------------+
| 104000000.0  |     0.0789    | 138800.0 |     0.77    |        0.0         |
| 1357000000.0 |     0.7121    | 271400.0 |    126.08   | 0.7047760455332839 |
| 470000000.0  |     0.0637    | 370300.0 |     1.15    | 0.2952239544667162 |
+--------------+---------------+----------+-------------+--------------------+
+---------------------+----------------------+---------------------+
|  balanced_amplitude |   balanced_change    |    balanced_close   |
+---------------------+----------------------+---------------------+
|  0.379495437466452  | 0.030952380952380953 | 0.10654338549075391 |
|  0.3397745571658615 |  0.8857142857142857  |  0.7116642958748223 |
| 0.28073000536768655 | 0.08333333333333333  | 0.18179231863442388 |
+---------------------+----------------------+---------------------+
+--------------------------+-----+
| balanced_income_increase | ... |
+--------------------------+-----+
|    0.2896345586216731    | ... |
|    0.5873629732164869    | ... |
|   0.12300246816184006    | ... |
+--------------------------+-----+
[3 rows x 27 columns]

# ----------新闻爬虫----------

In [ ]:
# 定义移除标点符号函数
def removePunctuation(contents):
    return re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "", contents)


# 分词函数
def splitWords(contents):
    segmentor = Segmentor()  # 初始化实例
    segmentor.load(cws_model_path)  # 加载模型
    words = segmentor.segment(contents)  # 分词
    segmentor.release()  # 释放模型
    return words


def tagWords(contents_split):
    postagger = Postagger() # 初始化实例
    postagger.load(pos_model_path)  # 加载模型

    words = contents_split  # 分词结果
    postags = postagger.postag(words)  # 词性标注

    postagger.release()  # 释放模型
    return postags


def getNews(stockName):
    hexStockName = str(stockName.encode('utf-8'))
    hexStockName = hexStockName.replace('\\', '%')
    hexStockName = hexStockName.replace('b\'', '')
    hexStockName = hexStockName.replace('\'', '')
    hexStockName = hexStockName.upper()
    hexStockName = hexStockName.replace('X', '')
    print(hexStockName)
    url = 'http://so.eastmoney.com/news/s?keyword='  + hexStockName
    # 爬取网站源代码
    browser = webdriver.Chrome()  # Get local session of chrome
    browser.get(url)  # Load page
    soup = BeautifulSoup(browser.page_source, "lxml")
    # browser.close()
    
    # 处理搜索undefined的情况
    if len(soup.findAll(class_ = 'still')) != 0:
        print(soup.findAll(target = 'module module-empty'))
        #url = soup.findAll(target = '_self')['href']
        browser = webdriver.Chrome()  # Get local session of chrome
        browser.get(url)  # Load page
        soup = BeautifulSoup(browser.page_source, "lxml")
        
    
    news_data = tc.SFrame({'code': ['000000'], 'name': ['哔哩哔哩'],
                          'url': ['http://www.bilibili.com'], 'title': ['News Title'],
                          'contents': ['news contents']})
    
    # 获取全部新闻的url
    news_url_list = []
    for tag in soup.findAll('a', href = True):
        if tag['href'][0:32] == 'http://stock.eastmoney.com/news/':
            news_url_list.append(tag['href'])
    print(news_url_list)  # Debug
    """
    # 运行移除函数并创建一个名为'contents_nopunc'的列
    news_data['contents_nopunc'] = news_data['contents'].apply(removePunctuation)
    # 执行分词函数
    news_data['content_split_words'] = news_data['contents_nopunc'].apply(splitWords)
    # 执行标注函数
    news_data['content_,tag_words'] = news_data['content_split_words'].apply(tagWords)
    """
    return news_data




In [ ]:
# print(getNews('润禾材料'))
getNews('润禾材料')

In [ ]:
news_list = []
for stock_name in SELECTED_DATA['name']:
    news_list.append(getNews(stock_name))

In [ ]:
browser = webdriver.Chrome()  # Get local session of chrome
browser.get('http://so.eastmoney.com/news/s?keyword=%E6%B6%A6%E7%A6%BE%E6%9D%90%E6%96%99')  # Load page
# soup = BeautifulSoup(browser.page_source, "lxml")
soup = BeautifulSoup(browser.page_source, "html.parser")

In [ ]:
# print(soup.find_all(class_ = 'still'))
# print(soup.find(class_ = 'still')['href'])
# print(soup.findAll(class_ = 'still',))
still = soup.find(class_ = 'still')
print(type(still('a')))